In [1]:
from bs4 import BeautifulSoup

import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import pandas as pd
import xmltodict
import requests
import os
import json
import numpy as np
from datetime import datetime, timedelta
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값

def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     

In [2]:
df_ttl = pd.DataFrame()
for target_keyword in ["미래에셋증권",'키움증권','삼성증권','NH투자증권']:
    new_html = requests.get('http://news.google.com/news?hl=ko&gl=kr&ie=UTF-8&output=rss&q=' + target_keyword )
    new_html.text
    soup = BeautifulSoup(new_html.text, 'html.parser')
    cnt = 0 
    news_title_l = []
    news_link_l = []
    news_writer_l = []
    url_l = []
    date_l = [] 

    for i in range(len(soup.findAll('item'))):
        news_title_l.append(soup.findAll('item')[i].find('title').text)
        news_link_l.append(str(soup.findAll('item')[i]).split('link/>')[1].split('<')[0])
        news_writer_l.append(    soup.findAll('item')[i].text.split(">")[-1])
        url_l.append(soup.findAll('item')[i].find('description').text.split('"')[1])
        date_l.append((datetime.strptime(soup.findAll('item')[i].find('pubdate').text, '%a, %d %b %Y %H:%M:%S GMT')+ timedelta(hours=9)).strftime('%Y%m%d-%H%M%S'))

        cnt +=1
        if cnt ==100:
            break

    df_ = pd.DataFrame()
    df_['news_title'] = news_title_l
    df_['news_link'] = news_link_l
    df_['news_writer'] = news_writer_l
    df_['news_url'] = url_l
    df_['date'] = date_l
    df_['CRAWLING_DATETIME'] = datetime.now().strftime('%Y%m%d-%H%M%S')


    df_['DATETIME'] = df_['date'] 
    df_['VIEWS'] = np.nan
    df_['LIKES'] = np.nan
    df_['TITLE'] =  df_['news_title']
    df_['CONTENT'] = np.nan
    df_['WRITER'] =df_['news_writer']

    df_['SOURCE'] = "구글뉴스"
    df_['RELATED_ITEM'] = target_keyword
    df_['POSITIVITY'] = np.nan
    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    df_['OTHERS'] = np.nan
    df_['NUM_REPLY'] = np.nan
    df_['CRAWLING_DATETIME'] = datetime.now().strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  df_['news_url'] 
    df_ttl = df_ttl.append(df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]])


/home/lsi8505/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/tmp/ipykernel_65248/1741130801.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ttl = df_ttl.append(df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]])
/home/lsi8505/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an H

In [3]:
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '구글뉴스' ORDER BY DATETIME desc",conn)
df_already

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,구글뉴스,키움증권,20220830-101923,"뮤직카우, 키움증권서 전략적 투자 유치 - ZD넷 코리아",None,0.044352,None,https://zdnet.co.kr/view/?no=20220830095327,20220830-012348,None
1,구글뉴스,키움증권,20220830-094937,"코스피, 뉴욕증시 하락에도 상승 출발 < 증권 < 금융/증권 < 기사본문 - 이코노믹리뷰",None,2.348617,None,https://www.econovill.com/news/articleView.htm...,20220830-012348,None
2,구글뉴스,미래에셋증권,20220830-094547,"[투자의 힘] 하나증권 ""간판 교체·ESG 모범생…초대형 IB 도약 준비"" - 뉴스웍스",None,74.278230,None,http://www.newsworks.co.kr/news/articleView.ht...,20220830-012340,None
3,구글뉴스,삼성증권,20220830-094547,"[투자의 힘] 하나증권 ""간판 교체·ESG 모범생…초대형 IB 도약 준비"" - 뉴스웍스",None,74.278230,None,http://www.newsworks.co.kr/news/articleView.ht...,20220830-012357,None
4,구글뉴스,NH투자증권,20220830-094547,"[투자의 힘] 하나증권 ""간판 교체·ESG 모범생…초대형 IB 도약 준비"" - 뉴스웍스",None,74.278230,None,http://www.newsworks.co.kr/news/articleView.ht...,20220830-012406,None
...,...,...,...,...,...,...,...,...,...,...
395,구글뉴스,삼성증권,20220208-170000,"낮에 하는 미국 주식 거래...삼성증권, '세계 최초' 서비스 오픈 - 한국경제매거진",None,71.434960,None,https://magazine.hankyung.com/business/article...,20220830-012357,None
396,구글뉴스,삼성증권,20220206-170000,"삼성증권 ""낮에도 미국 주식 사고파세요"" - 한국경제",None,0.197198,None,https://www.hankyung.com/finance/article/20220...,20220830-012357,None
397,구글뉴스,NH투자증권,20220121-170000,"NH투자증권, 창사 이래 첫 영업이익 1조원 달성 - 한국경제",None,99.955086,None,https://www.hankyung.com/finance/article/20220...,20220830-012406,None
398,구글뉴스,키움증권,20211220-170000,키움증권 - 한국경제,None,0.022628,None,https://www.hankyung.com/finance/article/20211...,20220830-012348,None


In [6]:
df_ttl = df_ttl[~df_ttl.index .isin(df_ttl.merge(df_already, on=['DATETIME','TITLE']).index)]
df_ttl = df_ttl.reset_index(drop=True)
df_ttl

In [8]:

if len(df_ttl) >0:

    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl
    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')  , " GOOGLE NEWS - SUCCESS  : ",len(df_ttl))

else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " GOOGLE NEWS - NO DATA TO UPDATE")


20220830-102654  GOOGLE NEWS - NO DATA TO UPDATE


In [10]:
df_ttl = pd.DataFrame()

target_keyword ="미래에셋증권"
new_html = requests.get('http://news.google.com/news?hl=ko&gl=kr&ie=UTF-8&output=rss&q=' + target_keyword )
new_html.text
soup = BeautifulSoup(new_html.text, 'html.parser')
cnt = 0 
news_title_l = []
news_link_l = []
news_writer_l = []
url_l = []
date_l = [] 

for i in range(len(soup.findAll('item'))):
    news_title_l.append(soup.findAll('item')[i].find('title').text)
    news_link_l.append(str(soup.findAll('item')[i]).split('link/>')[1].split('<')[0])
    news_writer_l.append(    soup.findAll('item')[i].text.split(">")[-1])
    url_l.append(soup.findAll('item')[i].find('description').text.split('"')[1])
    date_l.append((datetime.strptime(soup.findAll('item')[i].find('pubdate').text, '%a, %d %b %Y %H:%M:%S GMT')+ timedelta(hours=9)).strftime('%Y%m%d-%H%M%S'))

    cnt +=1
    if cnt ==8:
        break

df_ = pd.DataFrame()
df_['news_title'] = news_title_l
df_['news_link'] = news_link_l
df_['news_writer'] = news_writer_l
df_['news_url'] = url_l
df_['date'] = date_l
df_['CRAWLING_DATETIME'] = datetime.now().strftime('%Y%m%d-%H%M%S')


df_['DATETIME'] = df_['date'] 
df_['VIEWS'] = np.nan
df_['LIKES'] = np.nan
df_['TITLE'] =  df_['news_title']
df_['CONTENT'] = np.nan
df_['WRITER'] =df_['news_writer']

df_['SOURCE'] = "구글뉴스"
df_['RELATED_ITEM'] = target_keyword
df_['POSITIVITY'] = np.nan
for i in range(len(df_)):
    df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
df_['OTHERS'] = np.nan
df_['NUM_REPLY'] = np.nan
df_['CRAWLING_DATETIME'] = datetime.now().strftime('%Y%m%d-%H%M%S')
df_['URL'] =  df_['news_url'] 
df_ttl = df_ttl.append(df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]])
df_ttl

/home/lsi8505/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/tmp/ipykernel_65248/2693049961.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ttl = df_ttl.append(df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]])


,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,구글뉴스,미래에셋증권,20220829-110642,"미래에셋증권, 2분기 25개 증권사 중 '고객경영' 관심도 1위 - 뉴스저널리즘",NaN,84.995750,NaN,https://www.ngetnews.com/news/articleView.html...,20220831-010409,NaN
1,구글뉴스,미래에셋증권,20220830-132500,"한국거래소, 미래에셋증권 등 6개 증권사와 시장조성계약 - 뉴스핌",NaN,0.028818,NaN,https://www.newspim.com/news/view/20220830000690,20220831-010409,NaN
2,구글뉴스,미래에셋증권,20220809-160000,"미래에셋증권, 상반기 영업익 6059억원 - 한국경제",NaN,0.031968,NaN,https://www.hankyung.com/finance/article/20220...,20220831-010409,NaN
3,구글뉴스,미래에셋증권,20220830-070900,[소비자가 뽑은 참 좋은 브랜드-미래에셋증권] 퇴직연금 절대 강자...끊임없는 혁신...,NaN,99.954960,NaN,http://www.consumernews.co.kr/news/articleView...,20220831-010409,NaN
4,구글뉴스,미래에셋증권,20220831-084700,미래에셋증권(주) 상장지수증권 괴리율 초과 발생(미래에셋 코스닥150 선물 ETN)...,NaN,85.551056,NaN,http://vip.mk.co.kr/news/view/21/28/1016329.html,20220831-010409,NaN
5,구글뉴스,미래에셋증권,20220828-060000,[투자할 결심]① 서철수 미래에셋 리서치센터장 “글로벌 공급망 재편 과정은 한국 기...,NaN,0.354577,NaN,https://biz.chosun.com/stock/stock_general/202...,20220831-010409,NaN
6,구글뉴스,미래에셋증권,20220808-160000,"미래에셋증권 “롯데쇼핑, 실적 턴어라운드 기대… 목표가 상향” - 조선비즈 - 조선비즈",NaN,85.687355,NaN,https://biz.chosun.com/stock/stock_general/202...,20220831-010409,NaN
7,구글뉴스,미래에셋증권,20220830-182700,"미래에셋자산운용, 국내 최초 TIF ETF 출시…상품권 증정 이벤트 - 조선비즈 -...",NaN,1.475038,NaN,https://biz.chosun.com/stock/stock_general/202...,20220831-010409,NaN
